In [1]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from data_importer.base.storage import AzureStorage
from data_importer.base.cleaner import DataCleaner, strip, titleize
from data.constants import ACTIVE_UNKNOWN_CHOICE, ACTIVE_YES_CHOICE, ACTIVE_NO_CHOICE

In [2]:
storage = AzureStorage(account_name='cpdbv2data', 
                       account_key='your-key-here',
                       share='cleaned-data')

import_db = 'import'

current_status_dict = {
    '1.0': ACTIVE_YES_CHOICE,
    '0.0': ACTIVE_NO_CHOICE
}

## Read profiles data

In [3]:
df = pd.read_csv(
    storage.path_for('profiles/final-profiles.csv'),
    usecols=['UID', 'Current.Status'],
    converters={
        'Current.Status': lambda x: current_status_dict.get(x, ACTIVE_UNKNOWN_CHOICE)
    }
)

df = df.rename(columns={
    'UID': 'id',
    'Last.Name': 'last_name',
    'Current.Status': 'active'
})

In [4]:
df = df[0:32140]

## Update officer status

In [5]:
def update_officer_status(data):
    officer = Officer.objects.using(import_db).get(pk=data['id'])
    officer.active = data['active']
    officer.race = officer.race.title() if officer.race != "API" else officer.race
    officer.save()

In [6]:
tqdm.pandas(tqdm_notebook(), desc='Updating Officer Status')

df.progress_apply(update_officer_status, axis=1)
pd.DataFrame.from_records(Officer.objects.using(import_db).all().values())

A Jupyter Widget

Updating Officer Status: 32140it [02:08, 249.67it/s]          


,active,appointed_date,birth_year,first_name,gender,id,last_name,middle_initial,race,rank,suffix_name,tags
0,No,1969-01-06,1942.0,Carmen,M,5,Abbate,None,White,,None,[]
1,No,1954-10-16,1930.0,Anthony,M,6,Abbate,None,White,,None,[]
2,Yes,1995-12-04,1972.0,Terry,M,7,Abbate,None,White,,None,[]
3,No,1990-03-26,1945.0,Leon,M,8,Abbey,None,Black,,None,[]
4,No,1949-03-16,1923.0,Michael,M,9,Abbey,None,Black,,None,[]
5,No,1970-12-14,1949.0,Laura,M,11,Abbott,None,White,,None,[]
6,No,1982-11-29,1953.0,Andrew,M,12,Abbott,None,White,,None,[]
7,No,1987-12-07,1958.0,Dale,F,13,Abbott,None,White,,None,[]
8,No,1986-07-14,1965.0,Elizabeth,F,14,Abbott,None,White,,None,[]
9,Yes,2012-11-30,1983.0,Zaid,M,15,Abdallah,None,White,,None,[]
